In [84]:
import os
import struct
import numpy as np
import tensorflow as tf 
import csv
# from tensorflow.examples.tutorials.mnist import input_data

# CONV1_SIZE=5
# CONV1_KERNEL_NUM=32
# CONV2_SIZE=5
# CONV2_KERNEL_NUM=64

#LENGTH_OF_SEQUENCE
LENGTH=16
NUM_CHANNELS=1
CONV1_SIZE=1
CONV1_WIDTH=2
CONV1_KERNEL_NUM=1
CONV2_SIZE=1
CONV2_WIDTH=2
CONV2_KERNEL_NUM=1
FC_SIZE=512
OUTPUT_NODE=3
INPUT_NODE=784
BATCH_SIZE=210

LEARNING_RATE_BASE=0.005
LEARNING_RATE_DECAY=0.99
REGULARIZER=0.0001
STEPS=50000
MOVING_AVERAGE_DECAY=0.99
MODEL_SAVE_PATH="./model/"
MODEL_NAME="1d_model"

# 数据处理

In [55]:
#数据读取
file='../data/train_FD001.csv'
data=[]
with open(file,'r',encoding='utf-8') as csvFile:
        reader = csv.reader(csvFile)
        for line in reader:
            data.append(line)
# file=open(path,'r')
# data=file.readline()
print(len(data))

20631


In [56]:
#数据处理，处理后data_all的每一个元素都是一个列表，装着每一行数据
data_all=[]
for i in range(len(data)):
    temp=data[i][0].split(' ')[0:26]
    data_all.append(temp)
data_all=np.array(data_all)
data_all=data_all.astype(np.float)
print(len(data_all),len(data_all[0]),data_all[0])

20631 26 [ 1.00000e+00  1.00000e+00 -7.00000e-04 -4.00000e-04  1.00000e+02
  5.18670e+02  6.41820e+02  1.58970e+03  1.40060e+03  1.46200e+01
  2.16100e+01  5.54360e+02  2.38806e+03  9.04619e+03  1.30000e+00
  4.74700e+01  5.21660e+02  2.38802e+03  8.13862e+03  8.41950e+00
  3.00000e-02  3.92000e+02  2.38800e+03  1.00000e+02  3.90600e+01
  2.34190e+01]


In [57]:
#获取每个id对应的rul
rul=[]
for i in range(1,len(data_all)):
    if(data_all[i][0]!=data_all[i-1][0]):
        rul.append(data_all[i-1][1])
rul.append(data_all[len(data_all)-1][1])
print(len(rul))

100


In [58]:
#下采样（因为normal类数量过多，我这里先对他进行下采样，减少他对整体的影响。remain是对每个id的采样数量，后期你可以自己改）
data_new=[]
left=0
right=192
remain=30
for t in range(100):
    step=int((right-left-30)/remain)
    for j in range(left,right-30,step):
        data_new.append(data_all[j])
    for j in range(right-30,right):
        data_new.append(data_all[j])
    if(t!=99):
        left+=int(rul[t])
        right+=int(rul[t+1])
print(len(data_new))

6306


In [59]:
'''
Critical – If RUL (Remaining Useful Life) is less than 15 Hrs/15 
Cycles 
Warning – If RUL (Remaining Useful Life) is less than 30 Hrs/30
Cycles 
Normal- If RUL (Remaining Useful Life is) more than 30 Hrs/30 Cycles
'''
#按照上面的注释制作标签，critical对应2，warning对应1，normal对应0
label=[]
y_all=[]
for i in range(len(data_new)):
    count=int(data_new[i][0])
    if rul[count-1]-data_new[i][1]<15:
        label.append('Critical')
        y_all.append(2)
    elif rul[count-1]-data_new[i][1]<30:
        label.append('Warning')
        y_all.append(1)
    else:
        label.append('Normal')
        y_all.append(0)
print(len(label))

6306


In [ ]:
# print(label[0:200],y_all[0:200])

In [79]:
#我发现有好多列数据，从开始到最后都没有变过，因此我只取了发生变化的那几列（也许不变的数据也有用吧，只是我现在没有用到）
#之后是求均值和方差，归一化用
avg=[]
var=[]
data_new=np.array(data_new)
x_all=np.hstack((data_new[:,2:4],data_new[:,6:9],data_new[:,11:14],data_new[:,15:20],data_new[:,21:22],data_new[:,24:26]))

x_all=np.array(x_all)
x_min=np.amin(x_all,axis=0)
x_max=np.amax(x_all,axis=0)
# print(len(x_min),len(x_max))
print(x_all[0],x_all[1],x_all[:,8])
x_all=(x_all-x_min)/(x_max-x_min)
x_all=x_all-0.5
print(x_all[0])
# for i in range(17):
#     temp=x_all[:,i]
#     avg.append(np.mean(temp))
#     var.append(np.var(temp))
# print(len(avg),len(var))
# print(x_all[0],avg,var)

[-7.00000e-04 -4.00000e-04  6.41820e+02  1.58970e+03  1.40060e+03
  5.54360e+02  2.38806e+03  9.04619e+03  4.74700e+01  5.21660e+02
  2.38802e+03  8.13862e+03  8.41950e+00  3.92000e+02  3.90600e+01
  2.34190e+01] [-4.30000e-03 -1.00000e-04  6.42100e+02  1.58447e+03  1.39837e+03
  5.54670e+02  2.38802e+03  9.04968e+03  4.71600e+01  5.21680e+02
  2.38803e+03  8.13285e+03  8.41080e+00  3.91000e+02  3.89800e+01
  2.33669e+01] [47.47 47.16 47.15 ... 48.09 48.39 48.2 ]
[-0.0433526  -0.33333333 -0.33128834 -0.09319817 -0.24264439  0.27094017
 -0.26923077 -0.390245   -0.13473054  0.13326226 -0.34375    -0.3003922
 -0.14950884 -0.22727273  0.28632479  0.24534867]


In [80]:
y_label=np.zeros([len(y_all),3])
for i in range(len(y_all)):
    y_label[i][y_all[i]]=1
x_all=np.array(x_all)
print(y_label)

[[1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 ...
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]]


# 网络结构，输入数据为[BATCH_SIZE, LENGTH_OF_SEQUENCE]

In [81]:
#初始化w
def get_weight(shape,regularizer):
    w=tf.Variable(tf.truncated_normal(shape,stddev=0.1))
    if regularizer != None: tf.add_to_collection('losses',tf.contrib.layers.l2_regularizer(regularizer)(w))
    return w

#初始化b
def get_bias(shape):
    b=tf.Variable(tf.zeros(shape))
    return b

def conv2d(x,w):
    #tf只能输入四维张量，所以把数据扩张为四维
    input_3d = tf.expand_dims(x, 1)
#     input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    convolution_output=tf.nn.conv2d(input_4d,w,strides=[1,1,1,1],padding='SAME')
    #计算完成之后再压缩回二维
    conv_output_1d = tf.squeeze(convolution_output)
    return conv_output_1d

def max_pool_2x2(x):
    input_3d = tf.expand_dims(x, 1)
#     input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    pool_output=tf.nn.max_pool(input_4d,ksize=[1,1,2,1],strides=[1,1,2,1],padding='SAME')
    pool_output_1d = tf.squeeze(pool_output)
    return pool_output_1d

#前向传播
def forward(x,train,regularizer):
    conv1_w=get_weight([CONV1_SIZE,CONV1_WIDTH,NUM_CHANNELS,CONV1_KERNEL_NUM],regularizer)
    conv1_b=get_bias([CONV1_KERNEL_NUM])
    conv1=conv2d(x,conv1_w)
    input_3d = tf.expand_dims(conv1, 1)
#     input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    relu1=tf.nn.relu(tf.nn.bias_add(input_4d,conv1_b))
    relu1=tf.squeeze(relu1)
    pool1=max_pool_2x2(relu1)
    
    conv2_w=get_weight([CONV2_SIZE,CONV2_WIDTH,CONV1_KERNEL_NUM,CONV2_KERNEL_NUM],regularizer)
    conv2_b=get_bias([CONV2_KERNEL_NUM])
    conv2=conv2d(pool1,conv2_w)
    input_3d = tf.expand_dims(conv2, 1)
#     input_3d = tf.expand_dims(input_2d, 0)
    input_4d = tf.expand_dims(input_3d, 3)
    relu2=tf.nn.relu(tf.nn.bias_add(input_4d,conv2_b))
    relu2=tf.squeeze(relu2)
    pool2=max_pool_2x2(relu2)
    
    pool_shape=pool2.get_shape().as_list()
    print(pool_shape)
    nodes=pool_shape[1]
    reshaped=tf.reshape(pool2,[pool_shape[0],nodes])
    
    fc1_w=get_weight([nodes,FC_SIZE],regularizer)
    fc1_b=get_bias([FC_SIZE])
    fc1=tf.nn.relu(tf.matmul(reshaped,fc1_w)+fc1_b)
    if train: fc1=tf.nn.dropout(fc1,0.5)
    
    fc2_w=get_weight([FC_SIZE,OUTPUT_NODE],regularizer)
    fc2_b=get_bias([OUTPUT_NODE])
    fc2=tf.matmul(fc1,fc2_w)+fc2_b
    
    return fc2

In [85]:
def backward():
    x=tf.placeholder(
        tf.float32,[
            BATCH_SIZE,
            LENGTH
        ]
    )
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE])
    y=forward(x,True,REGULARIZER)
    global_step=tf.Variable(0,trainable=False)
    
    ce=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_,1))
    cem=tf.reduce_mean(ce)
    loss=cem + tf.add_n(tf.get_collection('losses'))
    
    correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
    learning_rate=tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        200,
        LEARNING_RATE_DECAY,
        staircase=True
    )
    
    train_step=tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)
    
    ema = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    ema_op=ema.apply(tf.trainable_variables())
    with tf.control_dependencies([train_step,ema_op]):
        train_op=tf.no_op(name='train')
        
    saver=tf.train.Saver()
    
    with tf.Session() as sess:
        init_op=tf.global_variables_initializer()
        sess.run(init_op)
        
        for i in range(STEPS):
            nn=i%30
            #输入数据为x_all,标签为y_label，每次取一个batch的数据训练
            xs=x_all[nn*BATCH_SIZE:(nn+1)*BATCH_SIZE]
            ys=y_label[nn*BATCH_SIZE:(nn+1)*BATCH_SIZE]
#             xs,ys=mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs=np.reshape(xs,(
                BATCH_SIZE,
                LENGTH
            ))
            _,loss_value,step,acc=sess.run([train_op,loss,global_step,accuracy], feed_dict={x:reshaped_xs,y_:ys})
            if i%1000 == 0:
                print("After %d training steps, loss on training batch is %g. the acc is %f"% (step, loss_value,acc))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step=global_step)
                
def main():
    backward()
    
if __name__ == '__main__':
    main()

[210, 4]
After 1 training steps, loss on training batch is 1.17385. the acc is 0.633333
After 1001 training steps, loss on training batch is 1.05741. the acc is 0.571429
After 2001 training steps, loss on training batch is 1.07752. the acc is 0.528571
After 3001 training steps, loss on training batch is 0.981813. the acc is 0.571429
After 4001 training steps, loss on training batch is 0.849218. the acc is 0.704762
After 5001 training steps, loss on training batch is 0.674928. the acc is 0.742857
After 6001 training steps, loss on training batch is 0.516142. the acc is 0.819048
After 7001 training steps, loss on training batch is 0.509341. the acc is 0.828571
After 8001 training steps, loss on training batch is 0.536271. the acc is 0.804762
After 9001 training steps, loss on training batch is 0.431186. the acc is 0.847619
After 10001 training steps, loss on training batch is 0.446601. the acc is 0.833333
After 11001 training steps, loss on training batch is 0.494176. the acc is 0.819048

KeyboardInterrupt: 

In [49]:
x=np.random.rand(100,10)
x=x.astype(np.float32)
# x=tf.placeholder(tf.float32,[1,1,25,1])
# print(forward(x,True,REGULARIZER))
x_input_1d=tf.placeholder(tf.float32,[100,10])
y=forward(x_input_1d,True,REGULARIZER)
feed_dict = {x_input_1d: x}
with tf.Session() as sess:
        init_op=tf.global_variables_initializer()
        sess.run(init_op)
        print(sess.run(y, feed_dict=feed_dict))

[100, 3]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 